# Individuals who used the internet for interaction with public authorities

In [1]:
import pandas as pd
from functools import reduce #necessary to create the last dataframe
df = pd.read_csv(r"C:\Users\hp pc\Desktop\Tesi\Smart cities\Databases\3. Citizen Engagement\Public Participation and Democracy\isoc_r_gov_i.csv", encoding = "ISO-8859-1",index_col=0)
del df["UNIT"] #Delete the unit of measurement
# Rename cities in English. This Dataframe contains regional data
df = df.replace(["Attiki"],"Athens")
df = df.replace(['Région de Bruxelles-Capitale / Brussels Hoofdstedelijk Gewest'],'Bruxelles')
df = df.replace(['Hovedstaden'],'Copenhagen')
df = df.replace(['Helsinki-Uusimaa'],'Helsinki')
df = df.replace(['Comunidad de Madrid'],'Madrid')
df = df.replace(['Île de France'],'Paris')
df = df.replace(['Praha'],'Prague')
df = df.replace(['Lazio'],'Rome')
df = df.replace(['Stockholm'],'Stockholm')
df = df.replace(['Makroregion Województwo Mazowieckie'],'Warsaw')
df = df.replace(['Wien'],'Vienna')
df = df.replace(['Hrvatska'],'Zagreb')

#these are the cities that we analyse in our study
cities=["Athens","Bruxelles","Copenhagen", "Helsinki", "Madrid", "Paris", "Prague", "Rome", "Stockholm",
        "Warsaw","Vienna", "Zagreb"]
df = df.rename(columns={"GEO": "CITIES"}) # RENAME COLUMNS
df.sort_values(by=['CITIES'], inplace=True, ascending=True) # sort by cities
df["TIME"]=df.index # Create a column with the date, which is set as index by default
df.reset_index(drop=True, inplace=True)  # reset index
indicators = list(dict.fromkeys(df["INDIC_IS"])) # make list containing names of the variables (no duplicates)

unav_cities=[] # create an empty list that will be filled with cities which don't have available data
dfList=[] #create an empty list that will filled with dataframes
for nome_indicatore in indicators:
    df1=df[df.INDIC_IS == nome_indicatore] 
    for city in cities: 
        if df1[df1['CITIES'].str.contains(city)].empty: #if one city in our sample is not in the df:
            unav_cities.append(((city +" has no data for " + nome_indicatore))) # put the name of the city in the list of cities with data NA
    #create a subdataframe for each indicator
        else:
            sub_df=df1[df1.CITIES == city] #take the subdataframe with the name of each city 
            sub_df=sub_df[sub_df.TIME==max(sub_df["TIME"])] # get the row where date is the most recent
            dfList.append(sub_df)
df=(pd.concat(dfList,ignore_index=False)) # This df containes all variables: it needs to be splitted...

final_dfs=[]     #...split the previous df and creating the complete one
for indicator in indicators:
    df1=df[df.INDIC_IS == indicator] 
    df1 = df1.rename(columns={"Value": indicator}) #rename "Value" with the name of the variable 
    del df1["INDIC_IS"] # delete the column filled with the name of the variable. 
    df1.reset_index(drop=True, inplace=True) 
    df1.sort_values("CITIES", axis=0, ascending=True, inplace=True)
    final_dfs.append(df1) # this is the list of df that need to be merged
df= reduce(lambda x, y: pd.merge(x, y, on = 'CITIES'), final_dfs) #merge dfs on CITIES one next to the other
del df["TIME_y"] # (before deleting check if columns are equals)
del df["TIME_x"]
df


,CITIES,Internet use: interaction with public authorities (last 12 months),Internet use: submitting completed forms (last 12 months)
0,Athens,61,33
1,Bruxelles,59,39
2,Copenhagen,94,77
3,Helsinki,93,80
4,Madrid,67,54
5,Paris,78,68
6,Prague,67,39
7,Rome,26,17
8,Stockholm,86,79
9,Vienna,71,47


In [2]:
# RENAME COLUMNS USING EASIER NAMES and 
df.columns=["CITIES","inter_pub_aut","sub_comp_forms"]

# Apply min-max normalization formula to each value in df 
df["inter_pub_aut"]=[round(((s-min(df.inter_pub_aut))/(max(df.inter_pub_aut)-min(df.inter_pub_aut)))*10,1) for s in df["inter_pub_aut"]]
df["sub_comp_forms"]=[round(((s-min(df.sub_comp_forms))/(max(df.sub_comp_forms)-min(df.sub_comp_forms)))*10,1) for s in df["sub_comp_forms"]]

df1=df
df1

,CITIES,inter_pub_aut,sub_comp_forms
0,Athens,5.1,2.5
1,Bruxelles,4.9,3.5
2,Copenhagen,10.0,9.5
3,Helsinki,9.9,10.0
4,Madrid,6.0,5.9
5,Paris,7.6,8.1
6,Prague,6.0,3.5
7,Rome,0.0,0.0
8,Stockholm,8.8,9.8
9,Vienna,6.6,4.8


# Voters turnout + Gender equality

In [3]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
df = pd.read_excel(r"C:\Users\hp pc\Desktop\Tesi\Smart cities\Databases\3. Citizen Engagement\Public Participation and Democracy\Elections Turnout.xlsx", sheetname='Foglio1') 
df.columns=["CITIES","eu_turn","nat_turn","loc_turn","%_wom_city_council"]
df_turnout=df
df_turnout

C:\Users\hp pc\Documents\Python-Anaconda\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


,CITIES,eu_turn,nat_turn,loc_turn,%_wom_city_council
0,Athens,45.60,51.05,43.05,36.73
1,Bruxelles,81.61,81.68,82.85,46.94
2,Copenhagen,68.80,84.54,61.87,47.30
3,Helsinki,51.50,77.70,61.80,48.23
4,Madrid,67.87,74.54,68.23,47.36
5,Paris,57.88,56.67,56.26,50.92
6,Prague,38.31,67.13,46.44,21.53
7,Rome,48.91,71.68,57.03,46.94
8,Stockholm,63.03,87.32,82.13,50.00
9,Vienna,58.71,72.00,74.74,37.00


In [4]:
#call df with women proportion in cities
%store -r dfpopulation_cities
df_wom_pop = dfpopulation_cities[["CITIES", "%_women_pop",]]
df = pd.merge(df_turnout, df_wom_pop, on=['CITIES'])
#get absolute value of the percentage of women in the pop. and women in the city council
df["abs_value"]=abs(df["%_women_pop"]-df["%_wom_city_council"])
df

,CITIES,eu_turn,nat_turn,loc_turn,%_wom_city_council,%_women_pop,abs_value
0,Athens,45.60,51.05,43.05,36.73,52.53,15.80
1,Bruxelles,81.61,81.68,82.85,46.94,50.88,3.94
2,Copenhagen,68.80,84.54,61.87,47.30,50.62,3.32
3,Helsinki,51.50,77.70,61.80,48.23,52.53,4.30
4,Madrid,67.87,74.54,68.23,47.36,53.46,6.10
5,Paris,57.88,56.67,56.26,50.92,52.10,1.18
6,Prague,38.31,67.13,46.44,21.53,51.12,29.59
7,Rome,48.91,71.68,57.03,46.94,52.67,5.73
8,Stockholm,63.03,87.32,82.13,50.00,50.57,0.57
9,Vienna,58.71,72.00,74.74,37.00,51.86,14.86


In [5]:
# del useless comun and apply
del df["%_wom_city_council"]
del df["%_women_pop"]

#Apply normalization
df["eu_turn"]=[round(((s-min(df.eu_turn))/(max(df.eu_turn)-min(df.eu_turn)))*10,2) for s in df["eu_turn"]]
df["nat_turn"]=[round(((s-min(df.nat_turn))/(max(df.nat_turn)-min(df.nat_turn)))*10,2) for s in df["nat_turn"]]
df["loc_turn"]=[round(((s-min(df.loc_turn))/(max(df.loc_turn)-min(df.loc_turn)))*10,2) for s in df["loc_turn"]]

#Apply reverse formula for the absolute value: the higher the differest, the worse the index
df["gender_rep"]=[round(((max(df.abs_value)-s)/(max(df.abs_value)-min(df.abs_value)))*10,2) for s in df["abs_value"]]

del df["abs_value"]
df2=df
df2

,CITIES,eu_turn,nat_turn,loc_turn,gender_rep
0,Athens,3.04,0.00,0.00,4.75
1,Bruxelles,10.00,8.44,10.00,8.84
2,Copenhagen,7.53,9.23,4.73,9.05
3,Helsinki,4.18,7.35,4.71,8.71
4,Madrid,7.35,6.48,6.33,8.09
5,Paris,5.42,1.55,3.32,9.79
6,Prague,1.63,4.43,0.85,0.00
7,Rome,3.68,5.69,3.51,8.22
8,Stockholm,6.41,10.00,9.82,10.00
9,Vienna,5.58,5.78,7.96,5.08


# Political Participation on line 

In [6]:
import pandas as pd
from functools import reduce #necessary to create the last dataframe
df = pd.read_csv(r"C:\Users\hp pc\Desktop\Tesi\Smart cities\Databases\3. Citizen Engagement\Public Participation and Democracy\isoc_r_iuse_i.csv", encoding = "ISO-8859-1",index_col=0)
del df["UNIT"] #Delete the unit of measurement
# Rename cities in English. This Dataframe contains regional data
df = df.replace(["Attiki"],"Athens")
df = df.replace(['Région de Bruxelles-Capitale / Brussels Hoofdstedelijk Gewest'],'Bruxelles')
df = df.replace(['Hovedstaden'],'Copenhagen')
df = df.replace(['Helsinki-Uusimaa'],'Helsinki')
df = df.replace(['Comunidad de Madrid'],'Madrid')
df = df.replace(['Île de France'],'Paris')
df = df.replace(['Praha'],'Prague')
df = df.replace(['Lazio'],'Rome')
df = df.replace(['Stockholm'],'Stockholm')
df = df.replace(['Region Centralny (NUTS 2013)'],'Warsaw')
df = df.replace(['Wien'],'Vienna')
df = df.replace(['Hrvatska'],'Zagreb')

#these are the cities that we analyse in our study
cities=["Athens","Bruxelles","Copenhagen", "Helsinki", "Madrid", "Paris", "Prague", "Rome", "Stockholm",
        "Warsaw","Vienna", "Zagreb"]
df = df.rename(columns={"GEO": "CITIES"}) # RENAME COLUMNS
df.sort_values(by=['CITIES'], inplace=True, ascending=True) # sort by cities
df.reset_index(drop=True, inplace=True)  # reset index

df = df.rename(columns={"Value":"pol_part_online"}) # RENAME COLUMNS
del df["INDIC_IS"]

df["pol_part_online"]=[round(((s-min(df.pol_part_online))/(max(df.pol_part_online)-min(df.pol_part_online)))*10,2) for s in df["pol_part_online"]]
df3=df
df3

,CITIES,pol_part_online
0,Athens,1.74
1,Bruxelles,1.30
2,Copenhagen,10.00
3,Helsinki,4.78
4,Madrid,5.65
5,Paris,2.17
6,Prague,0.00
7,Rome,3.04
8,Stockholm,8.70
9,Vienna,1.74


# CREATE THE PUBLIC PARTICIPATION AND DEMOCRACY INDEX

In [7]:
df1

,CITIES,inter_pub_aut,sub_comp_forms
0,Athens,5.1,2.5
1,Bruxelles,4.9,3.5
2,Copenhagen,10.0,9.5
3,Helsinki,9.9,10.0
4,Madrid,6.0,5.9
5,Paris,7.6,8.1
6,Prague,6.0,3.5
7,Rome,0.0,0.0
8,Stockholm,8.8,9.8
9,Vienna,6.6,4.8


In [8]:
df2

,CITIES,eu_turn,nat_turn,loc_turn,gender_rep
0,Athens,3.04,0.00,0.00,4.75
1,Bruxelles,10.00,8.44,10.00,8.84
2,Copenhagen,7.53,9.23,4.73,9.05
3,Helsinki,4.18,7.35,4.71,8.71
4,Madrid,7.35,6.48,6.33,8.09
5,Paris,5.42,1.55,3.32,9.79
6,Prague,1.63,4.43,0.85,0.00
7,Rome,3.68,5.69,3.51,8.22
8,Stockholm,6.41,10.00,9.82,10.00
9,Vienna,5.58,5.78,7.96,5.08


In [9]:
df3

,CITIES,pol_part_online
0,Athens,1.74
1,Bruxelles,1.30
2,Copenhagen,10.00
3,Helsinki,4.78
4,Madrid,5.65
5,Paris,2.17
6,Prague,0.00
7,Rome,3.04
8,Stockholm,8.70
9,Vienna,1.74


In [10]:
df = pd.merge(df1, df2, on=['CITIES'])
df = pd.merge(df, df3, on=['CITIES'])
#CREATE LAST COLUMN
df['Citizen_Engagement_Index'] = round(df.mean(axis=1),2)
df

,CITIES,inter_pub_aut,sub_comp_forms,eu_turn,nat_turn,loc_turn,gender_rep,pol_part_online,Citizen_Engagement_Index
0,Athens,5.1,2.5,3.04,0.00,0.00,4.75,1.74,2.45
1,Bruxelles,4.9,3.5,10.00,8.44,10.00,8.84,1.30,6.71
2,Copenhagen,10.0,9.5,7.53,9.23,4.73,9.05,10.00,8.58
3,Helsinki,9.9,10.0,4.18,7.35,4.71,8.71,4.78,7.09
4,Madrid,6.0,5.9,7.35,6.48,6.33,8.09,5.65,6.54
5,Paris,7.6,8.1,5.42,1.55,3.32,9.79,2.17,5.42
6,Prague,6.0,3.5,1.63,4.43,0.85,0.00,0.00,2.34
7,Rome,0.0,0.0,3.68,5.69,3.51,8.22,3.04,3.45
8,Stockholm,8.8,9.8,6.41,10.00,9.82,10.00,8.70,9.08
9,Vienna,6.6,4.8,5.58,5.78,7.96,5.08,1.74,5.36


In [12]:
Citizen_Engagement_Index=df[["CITIES","Citizen_Engagement_Index"]]
Citizen_Engagement_Index 

,CITIES,Citizen_Engagement_Index
0,Athens,2.45
1,Bruxelles,6.71
2,Copenhagen,8.58
3,Helsinki,7.09
4,Madrid,6.54
5,Paris,5.42
6,Prague,2.34
7,Rome,3.45
8,Stockholm,9.08
9,Vienna,5.36


In [13]:
%store Citizen_Engagement_Index

Stored 'Citizen_Engagement_Index' (DataFrame)
